In [1]:
!pip install -U ../input/kerasapplications/Keras_Applications-1.0.8-py3-none-any.whl
!pip install ../input/qubvel/efficientnet-1.0.0-py3-none-any.whl
!pip install ../input/qubvel/image_classifiers-1.0.0-py3-none-any.whl
!pip install ../input/qubvel/segmentation_models-1.0.0-py3-none-any.whl

Processing /kaggle/input/kerasapplications/Keras_Applications-1.0.8-py3-none-any.whl
Processing /kaggle/input/qubvel/efficientnet-1.0.0-py3-none-any.whl
Processing /kaggle/input/qubvel/image_classifiers-1.0.0-py3-none-any.whl
Processing /kaggle/input/qubvel/segmentation_models-1.0.0-py3-none-any.whl


In [2]:
import gc
import os
import cv2
import zipfile
import rasterio
import numpy as np
import pandas as pd
from PIL import Image
import tifffile as tiff
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from rasterio.windows import Window
from torch.utils.data import Dataset
import segmentation_models as sm
import tensorflow as tf
import math
sm.set_framework('tf.keras')
sm.framework()

Segmentation Models: using `keras` framework.


'tf.keras'

In [3]:
cfg = {
    'img_train': '../input/hubmap-organ-segmentation/test_images/',
    'test_df': '../input/hubmap-organ-segmentation/test.csv',
    'img_size': 2304,
    'patches': 4,
    'patch_size': 256,
    'mask_threshold': 0.5,
    'model': '../input/fpn-effneyb2-my-tiles-bce-dice-loss-256p/best_model.h5'
}
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss()
total_loss = dice_loss + 1 * focal_loss
model = tf.keras.models.load_model(cfg['model'], compile=False)#custom_objects={'dice_loss_plus_1binary_focal_loss': total_loss, 'iou_score':sm.losses.IOUScore(threshold=0.5), 'f1_score': sm.metrics.FScore(threshold=0.5)}

2022-09-18 16:47:33.859790: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-18 16:47:33.861056: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-18 16:47:33.861816: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-18 16:47:33.862670: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [4]:
def generate_tile(img, slices=cfg['patches']):
    '''
    Generates tiles of images and its corresponding mask
    Returns: Two lists, image tiles and mask tiles of length slices**2 
    '''
    factor = int(img.shape[0] / slices)
    #print(f'Size of each frame/tile: {factor, factor}')
    img_tiles = []
    temp_img = None
    for x in range(slices):
        for y in range(slices):
            temp_img = img[factor*x : factor*(x+1), factor*y: factor*(y+1), ...]
            temp_img = cv2.resize(temp_img, (cfg['patch_size'], cfg['patch_size']), interpolation=cv2.INTER_AREA)
            temp_img = np.expand_dims(temp_img, axis=0)
            img_tiles.append(temp_img)
            del temp_img; gc.collect()
    return img_tiles

def rle2mask(mask_rle, shape:tuple):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def tiles2img(
    tiles: list, #list of mask tiles
    display = False
):
    '''
    Converts tiles of images and masks to one array
    '''
    slices = int(math.sqrt(len(tiles)))
    img = []
    temp_img = None
    for i in range(slices):
        temp_img = tiles[slices*i: slices*(i+1)]
        temp_img = np.hstack([i for i in temp_img])
        img.append(temp_img)
        del temp_img
    mask = np.vstack([i for i in img])
    if display:
        print(mask.shape)
        plt.imshow(mask, cmap='gray')
        plt.axis('off')
    return mask

def mask2rle(mask, orig_dim=160):
    #Rescale image to original size
    size = int(len(mask.flatten())**.5)
    n = Image.fromarray(mask.reshape((size, size))*255.0)
    n = n.resize((orig_dim, orig_dim))
    n = np.array(n).astype(np.float32)
    #Get pixels to flatten
    pixels = n.T.flatten()
    #Round the pixels using the half of the range of pixel value
    pixels = (pixels - min(pixels) > ((max(pixels) - min(pixels)) / 2)).astype(int)
    pixels = np.nan_to_num(pixels) #incase of zero-div-error
    
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0]
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

In [5]:
submissions = {'id':[], 'rle':[]}
df = pd.read_csv(cfg['test_df'])
TEST = '../input/hubmap-organ-segmentation/test_images/'
for _, row in tqdm(df.iterrows()):
    masks = []
    img_path = TEST + str(row['id']) + '.tiff'
    img = np.array(Image.open(img_path))
    if img.shape[0] > cfg['img_size']:
        img = cv2.resize(img, (cfg['img_size'], cfg['img_size']), interpolation=cv2.INTER_AREA)
        
    elif img.shape[0] < cfg['img_size']:
        img = cv2.resize(img, (cfg['img_size'], cfg['img_size']), interpolation=cv2.INTER_CUBIC)
        
    img_tiles = generate_tile(img); del img
    
    for i in range(len(img_tiles)):
        patch = img_tiles[i]; patch = patch / 255.0
        mask = model.predict(patch)
        mask = mask.reshape((cfg['patch_size'], cfg['patch_size'], 1))
        masks.append(mask)
    mask = tiles2img(masks)
    rle = mask2rle(np.round(mask), row['img_width'])
    #np.where(mask >= cfg['mask_threshold'], 1, 0)
    #np.round(mask).astype(np.float32)
    submissions['id'].append(row['id'])
    submissions['rle'].append(rle)

submissions = pd.DataFrame(submissions)
submissions.to_csv('submission.csv', index=False)
submissions

0it [00:00, ?it/s]

2022-09-18 16:47:45.652752: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-09-18 16:47:48.198075: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


,id,rle
0,10078,2046015 217 2048037 219 2050060 220 2052083 22...
